In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dataSet = pd.read_csv("digits.csv", header=None) #read the given csv
dataSet = dataSet.sample(frac=1).reset_index(drop=True) #randomize dataset

In [3]:
data = dataSet[dataSet.columns[:-1]] #delete the last column

In [4]:
classes=dataSet.iloc[:,-1].values #separate the class column

In [5]:
className=[i for i in range(10)] #define class names

In [6]:
classMean= pd.DataFrame(columns=className) #creating new dataFrame for class mean

In [7]:
classColumn=dataSet.columns[data.shape[1]] #index of columns in which the classes are specified
for i, rows in dataSet.groupby(classColumn): #group by the class column and take mean
    classMean[i]=rows.mean()
classMean.drop(classMean.tail(1).index, inplace=True) #delete the mean of the class column

In [8]:
withinClassCovariance=np.zeros((data.shape[1],data.shape[1])) #Creating Sw
for i, rows in dataSet.groupby(classColumn):
    rows=rows.drop([classColumn], axis=1)
    Sk=np.zeros((data.shape[1],data.shape[1]))
    for j, row in rows.iterrows():
        x, mi = row.values.reshape(data.shape[1],1), classMean[i].values.reshape(data.shape[1],1)
        Sk += (x - mi).dot((x - mi).T)
    withinClassCovariance += Sk

In [9]:
betweenClassCovariance=np.zeros((data.shape[1],data.shape[1])) #Creating SB
for i in classMean:
    n=len(dataSet.loc[dataSet[classColumn]==i].index)
    mc =  classMean[i].values.reshape(data.shape[1],1)
    m = data.mean().values.reshape(data.shape[1],1)
    betweenClassCovariance += n*(mc-m).dot((mc-m).T)

In [10]:
calcJ = np.dot(np.linalg.pinv(withinClassCovariance), betweenClassCovariance)
eigvals, eigvecs = np.linalg.eig(calcJ)
eiglist = [(eigvals[i], eigvecs[:, i]) for i in range(len(eigvals))]
eiglist = sorted(eiglist, key = lambda x : x[0], reverse = True)
eiglist
eigen_value_sums = sum(eigvals)
W=np.array([eiglist[i][1] for i in range (2)]).real #generate W matrix

In [11]:
LDA = np.array(data.dot(W.T)) #LDA output

In [12]:
def gaussian(x, mean, std):  #creating the Gaussian Model
    calc = 1. / ((2 * np.pi) ** (len(x) / 2.) * np.linalg.det(std) ** (-0.5))
    return calc * np.exp(-np.dot(np.dot((x - mean), np.linalg.inv(std)), (x - mean).T) / 2.)

In [13]:
def Likelihood(train,classes,projection, PiK, sigma, mu):  #likelihood function
    MLR = []
    for i in projection:
        MLRvalue = []
        for j in classes: 
            pdfValue = gaussian(i, mu[j], sigma[j])
            MLRvalue.append(PiK[j]*pdfValue)

        MLR.append(MLRvalue)
    return np.asarray(MLR)

In [14]:
def error(dataset): #function for error calculation
    data = dataset.drop([classColumn],axis=1)
    dataGroup = dataset.groupby(dataSet.iloc[:,-1])
    classes = dataGroup.groups.keys()
    
    trainData = {i:dataGroup.get_group(i) for i in classes}
    PiK = {}
    mu = {}
    sigma = {}
    for j in classes:
        classData = trainData[j].drop([classColumn], axis=1)
        projection = np.dot(W, classData.T).T
        PiK[j] = classData.shape[0]/dataSet.shape[0]
        mu[j] = np.mean(projection,axis=0)
        sigma[j] = np.cov(projection, rowvar = False)
    projection = np.dot(W, data.T).T
    likelihood = Likelihood(data,classes, projection,PiK,sigma,mu )
    labels = np.argmax(likelihood, axis=1)
    errors = np.sum(labels != dataset.iloc[:, classColumn])
    return errors

In [15]:
fold = 10
ratio = 1/fold
index_at = 0
testSize = math.ceil(data.shape[0] * 0.1)
errorList = []

for k in range(fold):
    trainTestData = dataSet

    trainData = {}
    testData  = {}

    testData = trainTestData.iloc[index_at: index_at + testSize]
    trainData = trainTestData.drop(dataSet.index[index_at: index_at + testSize])
    
    testErrors = error(testData)
    trainErrors = error(trainData)
    
    KFoldError = [trainErrors/len(trainData)*100.0, testErrors/len(testData)*100.0 ]
    errorList.append(KFoldError)

    index_at = index_at + testSize

In [16]:
errorHistory = pd.DataFrame(errorList, columns = ["Train Error(%)", "Test Error(%)"], index=list(range(1,11)))
errorMean=errorHistory.mean(axis=0)

In [17]:
print(errorHistory)
print(errorMean)

    Train Error(%)  Test Error(%)
1        31.663575      30.555556
2        31.045145      33.888889
3        33.580705      26.111111
4        31.416203      41.111111
5        31.849103      42.222222
6        32.220161      26.666667
7        32.714904      32.777778
8        32.405690      33.333333
9        33.395176      31.666667
10       33.641975      35.028249
Train Error(%)    32.393264
Test Error(%)     33.336158
dtype: float64
